<a href="https://colab.research.google.com/github/salkyna/Bitcoin-Price-Prediction/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We install mwclient library.


mwclient is a lightweight Python client library to the MediaWiki API which provides access to most API functionality.

In [2]:
!pip install mwclient

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import mwclient
import time

site = mwclient.Site("en.wikipedia.org") #
page = site.pages["Bitcoin"]

In [4]:
revs = list(page.revisions())

In [5]:
revs[0]

OrderedDict([('revid', 1112934641),
             ('parentid', 1112815930),
             ('user', 'Jtbobwaysf'),
             ('timestamp',
              time.struct_time(tm_year=2022, tm_mon=9, tm_mday=28, tm_hour=21, tm_min=31, tm_sec=38, tm_wday=2, tm_yday=271, tm_isdst=-1)),
             ('comment',
              '/* 2020–present */ installed is not the correct word for this, its a network thus nothing is installed')])

In [6]:
revs = sorted(revs, key = lambda rev:rev["timestamp"])

In [7]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [10]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 2.1 MB/s 
     |████████████████████████████████| 163 kB 47.1 MB/s 
     |████████████████████████████████| 6.6 MB 49.6 MB/s 


In [11]:
import transformers

In [12]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [13]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [14]:
import statistics

In [15]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [16]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [17]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-08-05,1,0.748121,0.00
2009-08-06,2,0.995746,0.00
2009-08-14,1,0.930021,0.00
2009-10-13,2,-0.227501,0.50
...,...,...,...
2022-09-20,2,0.014269,0.50
2022-09-21,5,0.074705,0.40
2022-09-22,3,-0.997662,1.00
2022-09-27,1,0.748121,0.00


In [18]:
edits_df.index = pd.to_datetime(edits_df.index)

In [19]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end=datetime.today())

In [20]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [21]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-03-09,0,0.000000,0.00
2009-03-10,0,0.000000,0.00
2009-03-11,0,0.000000,0.00
2009-03-12,0,0.000000,0.00
...,...,...,...
2022-09-26,0,0.000000,0.00
2022-09-27,1,0.748121,0.00
2022-09-28,2,-0.999128,1.00
2022-09-29,0,0.000000,0.00


In [22]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [23]:
rolling_edits = rolling_edits.dropna()

In [24]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.133333,-0.018351,0.025000
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2022-09-26,0.833333,-0.130764,0.224444
2022-09-27,0.866667,-0.105826,0.224444
2022-09-28,0.866667,-0.134937,0.241111
2022-09-29,0.866667,-0.134937,0.241111


In [25]:
rolling_edits.to_csv("wikipedia_edits.csv")